In [1]:
import findspark
findspark.init('/home/kakade/spark')
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.appName("kakade").getOrCreate()

In [12]:
from pyspark.sql.types import StructField,StringType,FloatType,StructType
#Add header to data feature
feature_schema = [StructField("country", StringType(), True), StructField("sku_id", StringType(), True), \
                  StructField("title",StringType(), True), StructField("category_1",StringType(), True), \
                  StructField("category_2",StringType(), True), StructField("category_3",StringType(), True), \
                  StructField("description",StringType(), True), StructField("org_price", FloatType(), True), \
                  StructField("product_type",StringType(), True)]
# country sku_id title category_1 category_2 category_3 short_description price product_type 
feature_struc = StructType(fields=feature_schema)

In [13]:
train_feature = spark.read.csv("../Data/training/data_train.csv",escape='"',schema=feature_struc)

In [15]:
#most other category only contain <=5 products
#filter products out of the main categories
# train_feature.groupBy("category_1").count().orderBy("count",ascending=False).show(truncate=False)
main_category=train_feature.groupBy("category_1").count().filter("count>5")
train_feature = train_feature.join(main_category, col("train_feature.category_1") == col("main_category.category_1"), 'inner')

NameError: name 'col' is not defined

In [16]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover,CountVectorizer

In [17]:
def html_extract_func(col):
    soup=BeautifulSoup(col)
    return soup.get_text()
    return (1-col1/col2)**2
html_extract = udf(html_extract_func, StringType())
train_feature = train_feature.withColumn("html_extract",html_extract("description"))
# train_feature.select("html_extract").show(truncate=False)

In [18]:
train_feature.printSchema()

root
 |-- country: string (nullable = true)
 |-- sku_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- category_3: string (nullable = true)
 |-- description: string (nullable = true)
 |-- org_price: float (nullable = true)
 |-- product_type: string (nullable = true)
 |-- html_extract: string (nullable = true)



In [10]:
import pyspark.sql.functions as F
des_cat_test = train_feature.select("category_1","sku_id",F.regexp_replace(train_feature.html_extract, '(\d+)', ' ').alias('html_extract'))
regexTokenizer = RegexTokenizer(inputCol="html_extract", outputCol="description_token", pattern="\\W")
des_cat_test = regexTokenizer.transform(des_cat_test)
stop_words_remover = StopWordsRemover(inputCol="description_token", outputCol="stop_words_filtered")
des_cat_test = stop_words_remover.transform(des_cat_test)
des_cat_test.select("stop_words_filtered").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stop_words_filtered                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------

In [13]:
##currency_exchange
#set all price to PHP
import pyspark.sql.functions as F
from forex_python.converter import CurrencyRates
cex = CurrencyRates()
S2P=cex.get_rate("SGD","PHP")
M2P=cex.get_rate("MYR","PHP")
train_feature=train_feature.withColumn("price", F.when(train_feature.country == "my", M2P*train_feature.org_price).when(train_feature.country == "sg", S2P*train_feature.org_price).otherwise(train_feature.org_price))
                                    

In [15]:
train_feature.select("price","org_price").show()

+------------------+---------+
|             price|org_price|
+------------------+---------+
|            597.31|     49.0|
|           1560.32|    128.0|
| 305.6032962799072|    25.07|
|1438.4199999999998|    118.0|
|1399.4120372009277|    114.8|
|31681.809999999998|   2599.0|
| 4741.787980957031|   388.99|
|126.77599534988403|     10.4|
|               0.0|      0.0|
|            304.75|     25.0|
|115.56119441986084|     9.48|
| 950.8199999999999|     78.0|
|189.55450232505797|    15.55|
|6082.8099999999995|    499.0|
|            1462.8|    120.0|
|              null|     null|
| 1031.273981399536|     84.6|
|             365.7|     30.0|
| 203.0853981399536|    16.66|
|2072.2999999999997|    170.0|
+------------------+---------+
only showing top 20 rows



In [20]:
des_cat_test.groupBy("category_1").count().orderBy("count",ascending=False).show(truncate=False)

+--------------------------------------------------------------+-----+
|category_1                                                    |count|
+--------------------------------------------------------------+-----+
|Mobiles & Tablets                                             |7273 |
|Home & Living                                                 |6042 |
|Fashion                                                       |5729 |
|Watches Sunglasses Jewellery                                  |4216 |
|Health & Beauty                                               |4040 |
|Computers & Laptops                                           |2882 |
|TV, Audio / Video, Gaming & Wearables                         |2505 |
|Cameras                                                       |1950 |
|Home Appliances                                               |1583 |
| 8GB                                                          |5    |
| 4GB                                                          |5    |
|A1466

In [32]:
main_category=des_cat_test.groupBy("category_1").count().filter("count>5")
cat_train_test = des_cat_test.join(main_category,des_cat_test.category_1=main_category.category_1, 'inner')

SyntaxError: keyword can't be an expression (<ipython-input-32-8532e3f6520c>, line 2)

In [5]:
train_feature.select("sku_id").describe().show()

+-------+------------------+
|summary|            sku_id|
+-------+------------------+
|  count|             36283|
|   mean|              null|
| stddev|              null|
|    min|1A127ELAA3U0JKANPH|
|    max|  ZY795HLCZ3O0ANMY|
+-------+------------------+



In [2]:
train_data=spark.read.csv("../Clean_Data_Frame.csv",escape='"',header=True)

# train_data=spark.read.csv("/home/kakade/Support_files/Processed_Title_and_Description.csv",escape='"',header=True)

In [36]:
train_data.columns

['title', 'new_title', 'description', 'new_des', 'concise', 'clarity']

In [3]:
test_sample=train_data.filter(train_data["category_1"]=="Fashion").drop("category_1","org_price","clarity","country")
test_sample=test_sample.na.fill({"description":""})

In [11]:
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover,CountVectorizer
from pyspark.sql.types import StructField,StringType,FloatType,StructType,IntegerType
from pyspark.sql.functions import regexp_replace

def useful_token_func(col1,col2):
    if len(col1)>col2:
        return 0
    else:
        return col2-len(col1)
useful_token = udf(useful_token_func, IntegerType())

test_1 = test_sample.withColumn("title_no_sc1", regexp_replace(test_sample["title"], '[-/()]+', ' '))
test_1 = test_1.withColumn("title_no_sc", regexp_replace(test_1["title_no_sc1"], '''['.]+''', ''))

tokenizer = Tokenizer(inputCol="title_no_sc", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="title_no_sc", outputCol="regex_words", pattern="\\W")
stop_words_remover = StopWordsRemover(inputCol="regex_words", outputCol="no_stopwords")

countTokens = udf(lambda words: len(words), IntegerType())
charcount = udf(lambda x: len(x),IntegerType())

test_1=test_1.withColumn("title_character_count",charcount(test_1["title"]))
test_1.select("index","title_character_count").show()

test_1 = tokenizer.transform(test_1)
test_1=test_1.withColumn("tokens", countTokens(test_1["words"]))

test_1 = regexTokenizer.transform(test_1)
test_1=test_1.withColumn("regex_tokens", useful_token(test_1["regex_words"],test_1["tokens"]))

test_1 = stop_words_remover.transform(test_1)
test_1=test_1.withColumn("sw_tokens", useful_token(test_1["no_stopwords"],test_1["tokens"]))
test_1=test_1.withColumn("nsw_tokens", countTokens(test_1["no_stopwords"]))

+-----+---------------------+
|index|title_character_count|
+-----+---------------------+
|    1|                   46|
|   16|                   64|
|   27|                   53|
|   42|                  129|
|   43|                   90|
|   44|                   10|
|   45|                   73|
|   46|                   69|
|   47|                  140|
|   68|                   43|
|   72|                   72|
|   94|                   69|
|   95|                  128|
|   96|                   77|
|  104|                   97|
|  106|                   63|
|  107|                   63|
|  111|                   37|
|  114|                   46|
|  123|                   70|
+-----+---------------------+
only showing top 20 rows



In [5]:
test_1=test_1.drop("regex/org","sw/org","title_no_sc1")

In [6]:
test_2 = test_1.withColumn("desc_no_sc1", regexp_replace(test_sample["description"], '[-/()]+', ' '))
test_2 = test_2.withColumn("desc_no_sc", regexp_replace(test_2["desc_no_sc1"], '''['.]+''', ''))


regexTokenizer = RegexTokenizer(inputCol="desc_no_sc", outputCol="regex_des", pattern="\\W")
stop_words_remover = StopWordsRemover(inputCol="regex_des", outputCol="no_stopwords_des")

test_2 = regexTokenizer.transform(test_2)
test_2 = stop_words_remover.transform(test_2)
test_2=test_2.withColumn("des_tokens", countTokens(test_2["no_stopwords_des"]))
test_2=test_2.withColumn("r_tit_des",test_2["nsw_tokens"]/test_2["des_tokens"]).na.fill({"r_tit_des":1})

In [7]:
from pyspark.sql.types import ArrayType,StringType
uniqueExtract = udf(lambda words: list(set(words)),ArrayType(StringType()))
test_3=test_2.withColumn("unique_words",uniqueExtract(test_2["no_stopwords"]))
test_3=test_3.withColumn("count_repeat",countTokens(test_3["no_stopwords"])-test_3["nsw_tokens"])

In [13]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="unique_words", minSupport=0.1, minConfidence=0.5)
test_true=test_3.filter("concise==1")
fp_model = fpGrowth.fit(test_true)

# Display frequent itemsets.
fp_model.freqItemsets.show()

+---------+----+
|    items|freq|
+---------+----+
|  [shirt]| 285|
|  [women]| 379|
|  [dress]| 298|
|   [blue]| 312|
| [casual]| 287|
|   [intl]| 475|
|   [long]| 257|
| [sleeve]| 275|
|[fashion]| 319|
|   [mens]| 287|
|  [black]| 542|
+---------+----+



In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

string2int = udf(lambda x: int(x),IntegerType())

assembler = VectorAssembler(
    inputCols=["tokens", "nsw_tokens","sw_tokens", "regex_tokens","r_tit_des","count_repeat"],
    outputCol="features")
# assembler = VectorAssembler(
#     inputCols=["tokens","r_tit_des","count_repeat","sw_tokens"],
#     outputCol="features")


test_4 = assembler.transform(test_3)
test_4=test_4.select("index","title","features", "concise").withColumn("label",string2int(test_4["concise"]))

# test_3.filter("index==1").select("tokens","sw_tokens", "regex_tokens","r_tit_des","count_repeat").show()
# test_4.select("index","features", test_4["concise"].alias("label")).show()


In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

lr = LogisticRegression(maxIter=10)

evaluator = BinaryClassificationEvaluator()

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.7)
# Run TrainValidationSplit, and choose the best set of parameters.
tvsModel = tvs.fit(test_4)


In [23]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier

(trainingData, testData) = test_4.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

# Train model.  This also runs the indexers.
dt_model = dt.fit(trainingData)


# Select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator()

# Make predictions.

predictions = dt_model.transform(test_4)
accuracy = evaluator.evaluate(predictions)
print("Total Error = %g " % (1.0 - accuracy))

predictions = dt_model.transform(testData)
accuracy_test = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy_test))

Total Error = 0.22659 
Test Error = 0.246034 


In [24]:
# prediction = tvsModel.transform(test_4)
prediction = dt_model.transform(test_4)
accuracy = evaluator.evaluate(prediction)
print("Total Error = %g " % (1.0 - accuracy))

prediction = tvsModel.transform(testData)
accuracy_test = evaluator.evaluate(prediction)
print("Test Error = %g " % (1.0 - accuracy_test))

Total Error = 0.22659 
Test Error = 0.185717 


In [106]:
check_df=selected_df.filter("label==0").filter("prediction==1").select("index")
check_lr=selected_lr.filter("label==0").filter("prediction==1").select("index")
check_df.join(check_lr,"index","inner").describe().show()

+-------+------------------+
|summary|             index|
+-------+------------------+
|  count|               556|
|   mean|           18149.5|
| stddev|10861.372430741856|
|    min|             10057|
|    max|              9998|
+-------+------------------+



In [26]:
selected_df = predictions.select("index","title","label", "prediction")
selected_lr = prediction.select("index","title","label", "prediction")

['index', 'title', 'label', 'prediction']

In [33]:
selected_lr.drop("title").show(truncate=False)

+-----+-----+----------+
|index|label|prediction|
+-----+-----+----------+
|10005|1    |1.0       |
|10031|0    |0.0       |
|10057|0    |1.0       |
|10076|0    |1.0       |
|10148|0    |1.0       |
|10172|0    |0.0       |
|10200|0    |0.0       |
|10214|0    |0.0       |
|10222|1    |1.0       |
|10230|1    |1.0       |
|10252|1    |1.0       |
|10253|0    |1.0       |
|10260|0    |0.0       |
|1028 |0    |0.0       |
|1029 |0    |0.0       |
|10296|0    |0.0       |
|10299|0    |0.0       |
|10300|0    |0.0       |
|10339|1    |1.0       |
|10340|0    |0.0       |
+-----+-----+----------+
only showing top 20 rows



In [28]:
selected_lr.show(truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+
|index|title                                                                                                                                                      |label|prediction|
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------+
|10005|Halter Neck Backless Tank (EXPORT)                                                                                                                         |1    |1.0       |
|10031|Astar Coofandy Men's Warm Hooded Slim Pullover Coat Hoodies (Wine Red)                                                                                     |0    |0.0       |
|10057|New Style Hip Hop hat Street Snapbacks Caps(Export)                                     

In [15]:
test_true=test_3.filter("concise==1")
fp_model_true = fpGrowth.fit(test_true)

# Display frequent itemsets.
df_true=fp_model_true.freqItemsets

test_false=test_3.filter("concise==0")
fp_model_false = fpGrowth.fit(test_false)

# Display frequent itemsets.
df_false=fp_model_false.freqItemsets

df_true.join(df_false,"items","inner").show()

+---------+----+----+
|    items|freq|freq|
+---------+----+----+
|   [mens]| 287| 384|
| [casual]| 287| 634|
|  [dress]| 298| 374|
|   [long]| 257| 458|
|  [women]| 379|1215|
|   [intl]| 475| 784|
|  [black]| 542| 627|
| [sleeve]| 275| 391|
|   [blue]| 312| 326|
|[fashion]| 319| 805|
+---------+----+----+



In [21]:
test_sample.filter("concise==0").select("title").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------------+
|2016 New Arrive Women Four Seasons Flats Casual Shoes Women Flat Heel Cow Muscle Outsole Fashion Flat Women Genuine Leather Shoes             |
|Toprank Women Sexy Strapless Sleeveless Solid Bodycon Mini Dress Party Club Wear ( Khaki )                                                    |
|RUDY Dress                                                                                                                                    |
|Women Canvas Navy Style ID Credit Card Bag Girls Coin Bags Purse (Orange)                                                        

In [28]:
train_data.select("category_1").distinct().show(truncate=False)

+-------------------------------------+
|category_1                           |
+-------------------------------------+
|Computers & Laptops                  |
|Fashion                              |
|TV, Audio / Video, Gaming & Wearables|
|Mobiles & Tablets                    |
|Health & Beauty                      |
|Home Appliances                      |
|Watches Sunglasses Jewellery         |
|Cameras                              |
|Home & Living                        |
+-------------------------------------+



In [30]:
print("Computers & Laptops")
print(train_data.filter("category_1='''Computers & Laptops'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Computers & Laptops'''").filter("clarity==0").count())

print("TV, Audio / Video, Gaming & Wearables")
print(train_data.filter("category_1='''TV, Audio / Video, Gaming & Wearables'''").filter("clarity==1").count())
print(train_data.filter("category_1='''TV, Audio / Video, Gaming & Wearables'''").filter("clarity==0").count())

print("Mobiles & Tablets")
print(train_data.filter("category_1='''Mobiles & Tablets'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Mobiles & Tablets'''").filter("clarity==0").count())

print("Health & Beauty")
print(train_data.filter("category_1='''Health & Beauty'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Health & Beauty'''").filter("clarity==0").count())

print("Home Appliances")
print(train_data.filter("category_1='''Home Appliances'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Home Appliances'''").filter("clarity==0").count())

print("Watches Sunglasses Jewellery")
print(train_data.filter("category_1='''Watches Sunglasses Jewellery'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Watches Sunglasses Jewellery'''").filter("clarity==0").count())

print("Cameras")
print(train_data.filter("category_1='''Cameras'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Cameras'''").filter("clarity==0").count())

print("Home & Living")
print(train_data.filter("category_1='''Home & Living'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Home & Living'''").filter("clarity==0").count())

print("Fashion")
print(train_data.filter("category_1='''Fashion'''").filter("clarity==1").count())
print(train_data.filter("category_1='''Fashion'''").filter("clarity==0").count())


Computers & Laptops
2896
29
TV, Audio / Video, Gaming & Wearables
2495
16
Mobiles & Tablets
7239
38
Health & Beauty
3880
160
Home Appliances
1578
7
Watches Sunglasses Jewellery
3763
453
Cameras
1942
10
Home & Living
5803
245
Fashion
4632
1097


In [4]:
train_data.filter("category_1='''Fashion'''").select("category_3").distinct().show(truncate=False)

+------------------------+
|category_3              |
+------------------------+
|Bags                    |
|Muslim Wear             |
|Clothing                |
|Lingerie, Sleep & Lounge|
|Accessories             |
|Shoes                   |
|Wallets & Accessories   |
+------------------------+



In [ ]:
train_data.select("")